In [1]:
import findspark
findspark.init()

# PySpark Assignment

In [2]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('pyspark_assignment').getOrCreate()
spark

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

## 1. Basic DataFrame Operations

##### 1. Load the sales.csv and customer.csv files into separate DataFrames.

In [4]:
sales_df=spark.read.format('csv')\
                .option('header',True)\
                .option('inferSchema',True)\
                .load('sales.txt')
sales_df

DataFrame[sales_id: int, customer_id: int, product: string, amount: int, sale_date: date, region: string]

In [5]:
customer_df=spark.read.format('csv')\
                .option('header',True)\
                .option('inferSchema',True)\
                .load('customer.txt')
customer_df

DataFrame[customer_id: int, customer_name: string, email: string, age: int, city: string]

##### 2. Display the schema of both DataFrames.


In [6]:
sales_df.printSchema()

root
 |-- sales_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- amount: integer (nullable = true)
 |-- sale_date: date (nullable = true)
 |-- region: string (nullable = true)



In [7]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)



##### 3. Show the first 5 rows from the sales DataFrame.


In [8]:
sales_df.head(5)

[Row(sales_id=1, customer_id=101, product='Laptop', amount=50000, sale_date=datetime.date(2023, 1, 15), region='North'),
 Row(sales_id=2, customer_id=102, product='Mobile', amount=15000, sale_date=datetime.date(2023, 2, 10), region='South'),
 Row(sales_id=3, customer_id=103, product='Tablet', amount=20000, sale_date=datetime.date(2023, 3, 5), region='West'),
 Row(sales_id=4, customer_id=104, product='Laptop', amount=55000, sale_date=datetime.date(2023, 3, 15), region='East'),
 Row(sales_id=5, customer_id=105, product='Desktop', amount=40000, sale_date=datetime.date(2023, 4, 20), region='North')]

##### 4. Count the number of rows and columns in the customer DataFrame.


In [9]:
print('no. of rows =',customer_df.count())
print('no. of columns =',len(customer_df.columns))

no. of rows = 7
no. of columns = 5


## 2. Data Cleaning


##### 5. Remove duplicate rows from the sales DataFrame based on customer_id,product,amount,sale_date,region columns

In [10]:
sales_df=sales_df.dropDuplicates(['customer_id','product','amount','sale_date','region'])
sales_df.show(truncate=False)

+--------+-----------+-------+------+----------+------+
|sales_id|customer_id|product|amount|sale_date |region|
+--------+-----------+-------+------+----------+------+
|9       |104        |Desktop|45000 |2023-08-10|West  |
|3       |103        |Tablet |20000 |2023-03-05|West  |
|7       |102        |Laptop |60000 |2023-06-15|East  |
|6       |101        |Mobile |15000 |2023-05-10|South |
|1       |101        |Laptop |50000 |2023-01-15|North |
|10      |105        |Laptop |70000 |2023-09-25|North |
|4       |104        |Laptop |55000 |2023-03-15|East  |
|5       |105        |Desktop|40000 |2023-04-20|North |
|8       |103        |Tablet |20000 |2023-07-05|North |
|2       |102        |Mobile |15000 |2023-02-10|South |
+--------+-----------+-------+------+----------+------+



##### 6. Drop rows where any column in the customer DataFrame has null values.


In [11]:
customer_df.na.drop(how='any').show(truncate=False)

+-----------+-------------+---------------------+---+---------+
|customer_id|customer_name|email                |age|city     |
+-----------+-------------+---------------------+---+---------+
|101        |Arun Sharma  |arun.sharma@email.com|28 |Delhi    |
|102        |Meena Verma  |meena.verma@email.com|34 |Mumbai   |
|103        |Rahul Yadav  |rahul.yadav@email.com|30 |Bangalore|
|104        |Priya Patel  |priya.patel@email.com|27 |Ahmedabad|
|105        |Sneha Reddy  |sneha.reddy@email.com|29 |Hyderabad|
|106        |Vikas Jain   |vikas.jain@email.com |31 |Chennai  |
|107        |Amit Roy     |amit.roy@email.com   |35 |Kolkata  |
+-----------+-------------+---------------------+---+---------+



##### 7. Replace null values in the amount column of the sales DataFrame with 0.


In [12]:
sales_df.na.fill(0,'amount').show(truncate=False)

+--------+-----------+-------+------+----------+------+
|sales_id|customer_id|product|amount|sale_date |region|
+--------+-----------+-------+------+----------+------+
|9       |104        |Desktop|45000 |2023-08-10|West  |
|3       |103        |Tablet |20000 |2023-03-05|West  |
|7       |102        |Laptop |60000 |2023-06-15|East  |
|6       |101        |Mobile |15000 |2023-05-10|South |
|1       |101        |Laptop |50000 |2023-01-15|North |
|10      |105        |Laptop |70000 |2023-09-25|North |
|4       |104        |Laptop |55000 |2023-03-15|East  |
|5       |105        |Desktop|40000 |2023-04-20|North |
|8       |103        |Tablet |20000 |2023-07-05|North |
|2       |102        |Mobile |15000 |2023-02-10|South |
+--------+-----------+-------+------+----------+------+



##### 8. Replace null values in the email column of the customer DataFrame with the value "unknown".

In [13]:
customer_df.na.fill('unknown','email').show(truncate=False)

+-----------+-------------+---------------------+---+---------+
|customer_id|customer_name|email                |age|city     |
+-----------+-------------+---------------------+---+---------+
|101        |Arun Sharma  |arun.sharma@email.com|28 |Delhi    |
|102        |Meena Verma  |meena.verma@email.com|34 |Mumbai   |
|103        |Rahul Yadav  |rahul.yadav@email.com|30 |Bangalore|
|104        |Priya Patel  |priya.patel@email.com|27 |Ahmedabad|
|105        |Sneha Reddy  |sneha.reddy@email.com|29 |Hyderabad|
|106        |Vikas Jain   |vikas.jain@email.com |31 |Chennai  |
|107        |Amit Roy     |amit.roy@email.com   |35 |Kolkata  |
+-----------+-------------+---------------------+---+---------+



## 3. Column Manipulation


##### 9. Add a new column discounted_amount to the sales DataFrame that applies a 10% discount on amount.

In [14]:
sales_df.withColumn('discounted_amount',col('amount')-(col('amount')*0.1))\
.show(truncate=False)

+--------+-----------+-------+------+----------+------+-----------------+
|sales_id|customer_id|product|amount|sale_date |region|discounted_amount|
+--------+-----------+-------+------+----------+------+-----------------+
|9       |104        |Desktop|45000 |2023-08-10|West  |40500.0          |
|3       |103        |Tablet |20000 |2023-03-05|West  |18000.0          |
|7       |102        |Laptop |60000 |2023-06-15|East  |54000.0          |
|6       |101        |Mobile |15000 |2023-05-10|South |13500.0          |
|1       |101        |Laptop |50000 |2023-01-15|North |45000.0          |
|10      |105        |Laptop |70000 |2023-09-25|North |63000.0          |
|4       |104        |Laptop |55000 |2023-03-15|East  |49500.0          |
|5       |105        |Desktop|40000 |2023-04-20|North |36000.0          |
|8       |103        |Tablet |20000 |2023-07-05|North |18000.0          |
|2       |102        |Mobile |15000 |2023-02-10|South |13500.0          |
+--------+-----------+-------+------+-

##### 10. Rename the city column in the customer DataFrame to customer_city.


In [15]:
customer_df=customer_df.withColumnRenamed('city','customer_city')
customer_df.show(truncate=False)

+-----------+-------------+---------------------+---+-------------+
|customer_id|customer_name|email                |age|customer_city|
+-----------+-------------+---------------------+---+-------------+
|101        |Arun Sharma  |arun.sharma@email.com|28 |Delhi        |
|102        |Meena Verma  |meena.verma@email.com|34 |Mumbai       |
|103        |Rahul Yadav  |rahul.yadav@email.com|30 |Bangalore    |
|104        |Priya Patel  |priya.patel@email.com|27 |Ahmedabad    |
|105        |Sneha Reddy  |sneha.reddy@email.com|29 |Hyderabad    |
|106        |Vikas Jain   |vikas.jain@email.com |31 |Chennai      |
|107        |Amit Roy     |amit.roy@email.com   |35 |Kolkata      |
+-----------+-------------+---------------------+---+-------------+



##### 11. Drop the region column from the sales DataFrame.


In [16]:
sales_df.drop('region').show()

+--------+-----------+-------+------+----------+
|sales_id|customer_id|product|amount| sale_date|
+--------+-----------+-------+------+----------+
|       9|        104|Desktop| 45000|2023-08-10|
|       3|        103| Tablet| 20000|2023-03-05|
|       7|        102| Laptop| 60000|2023-06-15|
|       6|        101| Mobile| 15000|2023-05-10|
|       1|        101| Laptop| 50000|2023-01-15|
|      10|        105| Laptop| 70000|2023-09-25|
|       4|        104| Laptop| 55000|2023-03-15|
|       5|        105|Desktop| 40000|2023-04-20|
|       8|        103| Tablet| 20000|2023-07-05|
|       2|        102| Mobile| 15000|2023-02-10|
+--------+-----------+-------+------+----------+



##### 12. Create a new column customer_age_category in the customer DataFrame based on age:
    a. "Youth" for age < 30
    b. "Adult" for 30 <= age < 50
    c. "Senior" for age >= 50

In [17]:
customer_df.withColumn('customer_age_category',when(col('age')<30,'Youth')
                                            .when((col('age')>=30) & (col('age')<50),'Adult')
                                            .when(col('age')>=50,'Senior'))\
.show(truncate=False)

+-----------+-------------+---------------------+---+-------------+---------------------+
|customer_id|customer_name|email                |age|customer_city|customer_age_category|
+-----------+-------------+---------------------+---+-------------+---------------------+
|101        |Arun Sharma  |arun.sharma@email.com|28 |Delhi        |Youth                |
|102        |Meena Verma  |meena.verma@email.com|34 |Mumbai       |Adult                |
|103        |Rahul Yadav  |rahul.yadav@email.com|30 |Bangalore    |Adult                |
|104        |Priya Patel  |priya.patel@email.com|27 |Ahmedabad    |Youth                |
|105        |Sneha Reddy  |sneha.reddy@email.com|29 |Hyderabad    |Youth                |
|106        |Vikas Jain   |vikas.jain@email.com |31 |Chennai      |Adult                |
|107        |Amit Roy     |amit.roy@email.com   |35 |Kolkata      |Adult                |
+-----------+-------------+---------------------+---+-------------+---------------------+



## 4. Filtering


##### 13. Filter the sales DataFrame to show only rows where amount is greater than 50,000.


In [18]:
filtered_amount=sales_df.filter(col('amount')>50000)
filtered_amount.show()

+--------+-----------+-------+------+----------+------+
|sales_id|customer_id|product|amount| sale_date|region|
+--------+-----------+-------+------+----------+------+
|       7|        102| Laptop| 60000|2023-06-15|  East|
|      10|        105| Laptop| 70000|2023-09-25| North|
|       4|        104| Laptop| 55000|2023-03-15|  East|
+--------+-----------+-------+------+----------+------+



##### 14. Filter the customer DataFrame to show customers aged between 25 and 30.


In [19]:
# PySQL
customer_df.createOrReplaceTempView('customers')

spark.sql('''
        select customer_id,customer_name,email,age,customer_city
        from customers
        where age between 25 and 30 
''').show()

+-----------+-------------+--------------------+---+-------------+
|customer_id|customer_name|               email|age|customer_city|
+-----------+-------------+--------------------+---+-------------+
|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|        104|  Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|        105|  Sneha Reddy|sneha.reddy@email...| 29|    Hyderabad|
+-----------+-------------+--------------------+---+-------------+



##### 15. Identify all customers who have made purchases in more than one region.


In [20]:
sales_df.groupBy('customer_id')\
    .agg(count_distinct('region').alias('region_count'))\
    .filter(col('region_count')>1).show()

+-----------+------------+
|customer_id|region_count|
+-----------+------------+
|        101|           2|
|        103|           2|
|        102|           2|
|        104|           2|
+-----------+------------+



##### 16. Filter the top 3 sales based on amount for each product.


In [21]:
window=Window.partitionBy('product').orderBy(col('amount').desc())

sales_df.withColumn('rank',dense_rank().over(window)).filter(col('rank')<=3).show()

+--------+-----------+-------+------+----------+------+----+
|sales_id|customer_id|product|amount| sale_date|region|rank|
+--------+-----------+-------+------+----------+------+----+
|       9|        104|Desktop| 45000|2023-08-10|  West|   1|
|       5|        105|Desktop| 40000|2023-04-20| North|   2|
|      10|        105| Laptop| 70000|2023-09-25| North|   1|
|       7|        102| Laptop| 60000|2023-06-15|  East|   2|
|       4|        104| Laptop| 55000|2023-03-15|  East|   3|
|       6|        101| Mobile| 15000|2023-05-10| South|   1|
|       2|        102| Mobile| 15000|2023-02-10| South|   1|
|       3|        103| Tablet| 20000|2023-03-05|  West|   1|
|       8|        103| Tablet| 20000|2023-07-05| North|   1|
+--------+-----------+-------+------+----------+------+----+



## 5. Joins

##### 17. Perform an inner join between sales and customer DataFrames on customer_id.


In [22]:
sales_df.join(customer_df,sales_df.customer_id==customer_df.customer_id,'inner').show()

+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|sales_id|customer_id|product|amount| sale_date|region|customer_id|customer_name|               email|age|customer_city|
+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|       9|        104|Desktop| 45000|2023-08-10|  West|        104|  Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|       3|        103| Tablet| 20000|2023-03-05|  West|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|       7|        102| Laptop| 60000|2023-06-15|  East|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|       6|        101| Mobile| 15000|2023-05-10| South|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|       1|        101| Laptop| 50000|2023-01-15| North|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|      10|        105| Laptop| 7

##### 18. Perform a left join to include all records from sales and matching records from customer.


In [23]:
sales_df.join(customer_df,sales_df.customer_id==customer_df.customer_id,'left').show()

+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|sales_id|customer_id|product|amount| sale_date|region|customer_id|customer_name|               email|age|customer_city|
+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|       9|        104|Desktop| 45000|2023-08-10|  West|        104|  Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|       3|        103| Tablet| 20000|2023-03-05|  West|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|       7|        102| Laptop| 60000|2023-06-15|  East|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|       6|        101| Mobile| 15000|2023-05-10| South|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|       1|        101| Laptop| 50000|2023-01-15| North|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|      10|        105| Laptop| 7

##### 19. Perform a full outer join between sales and customer DataFrames.


In [24]:
sales_df.join(customer_df,sales_df.customer_id==customer_df.customer_id,'full_outer').show()

# done similar using SQL query
print('using PySQL.....')
customer_df.createOrReplaceTempView('customers')
sales_df.createOrReplaceTempView('sales')
spark.sql('''
    select *
    from sales s
    full join customers c on c.customer_id=s.customer_id
''').show()

+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|sales_id|customer_id|product|amount| sale_date|region|customer_id|customer_name|               email|age|customer_city|
+--------+-----------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|       6|        101| Mobile| 15000|2023-05-10| South|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|       1|        101| Laptop| 50000|2023-01-15| North|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|       7|        102| Laptop| 60000|2023-06-15|  East|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|       2|        102| Mobile| 15000|2023-02-10| South|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|       3|        103| Tablet| 20000|2023-03-05|  West|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|       8|        103| Tablet| 2

##### 20. Identify customers who have not made any purchases by performing an anti-join.


In [25]:
customer_df.join(sales_df,customer_df.customer_id==sales_df.customer_id,'left_anti').show()

+-----------+-------------+--------------------+---+-------------+
|customer_id|customer_name|               email|age|customer_city|
+-----------+-------------+--------------------+---+-------------+
|        106|   Vikas Jain|vikas.jain@email.com| 31|      Chennai|
|        107|     Amit Roy|  amit.roy@email.com| 35|      Kolkata|
+-----------+-------------+--------------------+---+-------------+



## 6. Aggregations


##### 21. Calculate the total sales amount for each product.


In [26]:
sales_df.groupBy(col('product')).agg(sum('amount').alias('total_amount')).show()

+-------+------------+
|product|total_amount|
+-------+------------+
| Laptop|      235000|
| Mobile|       30000|
| Tablet|       40000|
|Desktop|       85000|
+-------+------------+



##### 22. Find the average age of customers in the customer DataFrame.


In [27]:
customer_df.agg(avg(col('age')).alias('avg_age_of_customers')).show()

+--------------------+
|avg_age_of_customers|
+--------------------+
|  30.571428571428573|
+--------------------+



##### 23. Calculate the maximum and minimum sales amounts in the sales DataFrame.


In [28]:
sales_df.agg(min(col('amount')).alias('min_sale'),
             max(col('amount')).alias('max_sale'))\
        .show()

# using PySQL
print('using PySQL...')
sales_df.createOrReplaceTempView('sales')
spark.sql('''
        select min(amount) as min_sale,
                max(amount) as max_sale
        from sales
''').show()

+--------+--------+
|min_sale|max_sale|
+--------+--------+
|   15000|   70000|
+--------+--------+

using PySQL...
+--------+--------+
|min_sale|max_sale|
+--------+--------+
|   15000|   70000|
+--------+--------+



##### 24. Group the customer DataFrame by customer_city and count the number of customers in each city.

In [29]:
customer_df.groupBy('customer_city').agg(count_distinct('customer_id')
                                    .alias('customer_per_city'))\
.show()

+-------------+-----------------+
|customer_city|customer_per_city|
+-------------+-----------------+
|    Bangalore|                1|
|      Chennai|                1|
|       Mumbai|                1|
|    Ahmedabad|                1|
|      Kolkata|                1|
|        Delhi|                1|
|    Hyderabad|                1|
+-------------+-----------------+



## 7. Sorting

##### 25. Sort the sales DataFrame by amount in descending order.


In [30]:
sales_df.sort('amount',ascending=False).show()

+--------+-----------+-------+------+----------+------+
|sales_id|customer_id|product|amount| sale_date|region|
+--------+-----------+-------+------+----------+------+
|      10|        105| Laptop| 70000|2023-09-25| North|
|       7|        102| Laptop| 60000|2023-06-15|  East|
|       4|        104| Laptop| 55000|2023-03-15|  East|
|       1|        101| Laptop| 50000|2023-01-15| North|
|       9|        104|Desktop| 45000|2023-08-10|  West|
|       5|        105|Desktop| 40000|2023-04-20| North|
|       3|        103| Tablet| 20000|2023-03-05|  West|
|       8|        103| Tablet| 20000|2023-07-05| North|
|       6|        101| Mobile| 15000|2023-05-10| South|
|       2|        102| Mobile| 15000|2023-02-10| South|
+--------+-----------+-------+------+----------+------+



##### 26. Sort the customer DataFrame by age in ascending order.


In [31]:
customer_df.orderBy(col('age')).show()

+-----------+-------------+--------------------+---+-------------+
|customer_id|customer_name|               email|age|customer_city|
+-----------+-------------+--------------------+---+-------------+
|        104|  Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|        105|  Sneha Reddy|sneha.reddy@email...| 29|    Hyderabad|
|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|        106|   Vikas Jain|vikas.jain@email.com| 31|      Chennai|
|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|        107|     Amit Roy|  amit.roy@email.com| 35|      Kolkata|
+-----------+-------------+--------------------+---+-------------+



## 8. Union Operations


##### 27. Add a new dataset for customers and perform a union operation with the customer DataFrame.


In [32]:
new_customer_data = [
    [108, 'Pooja Joshi', 'pooja.joshi@email.com', 33, 'Chennai'],
    [109, 'Pooja Joshi', 'pooja.joshi@email.com', 35, 'Kolkata'],
    [110, 'Vikram Chauhan', 'vikram.chauhan@email.com', 34, 'Pune'],
    [111, 'Neha Gupta', 'neha.gupta@email.com', 35, 'Kolkata'],
    [112, 'Sunita Rao', 'sunita.rao@email.com', 28, 'Lucknow'],
    [113, 'Sunita Rao', 'sunita.rao@email.com', 33, 'Lucknow'],
    [114, 'Rakesh Bansal', 'rakesh.bansal@email.com', 28, 'Lucknow'],
    [115, 'Sunita Rao', 'sunita.rao@email.com', 25, 'Chennai']
]
customer_schema=['customer_id','customer_name','email','age','customer_city']
new_customer_df= spark.createDataFrame(data=new_customer_data,schema=customer_schema)


In [33]:
customer_df.union(new_customer_df).show()

+-----------+--------------+--------------------+---+-------------+
|customer_id| customer_name|               email|age|customer_city|
+-----------+--------------+--------------------+---+-------------+
|        101|   Arun Sharma|arun.sharma@email...| 28|        Delhi|
|        102|   Meena Verma|meena.verma@email...| 34|       Mumbai|
|        103|   Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|        104|   Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|        105|   Sneha Reddy|sneha.reddy@email...| 29|    Hyderabad|
|        106|    Vikas Jain|vikas.jain@email.com| 31|      Chennai|
|        107|      Amit Roy|  amit.roy@email.com| 35|      Kolkata|
|        108|   Pooja Joshi|pooja.joshi@email...| 33|      Chennai|
|        109|   Pooja Joshi|pooja.joshi@email...| 35|      Kolkata|
|        110|Vikram Chauhan|vikram.chauhan@em...| 34|         Pune|
|        111|    Neha Gupta|neha.gupta@email.com| 35|      Kolkata|
|        112|    Sunita Rao|sunita.rao@email.com

##### 28. Combine the sales DataFrame with another DataFrame containing additional sales records.

In [34]:
new_sales_data = [
    [11, 114, 'Tablet', 70000, '2023-09-13','North'],
    [12, 115, 'Mobile', 50000, '2023-11-16','West'],
    [13, 112, 'Tablet', 60000, '2023-01-04','South'],
    [14, 109, 'Desktop', 90000, '2023-03-08','East'],
    [15, 108, 'Laptop', 80000, '2023-06-25','East'],
    [16, 113, 'Tablet', 50000, '2023-01-18','North'],
    [17, 115, 'Desktop', 80000, '2023-02-05','North'],
    [18, 112, 'Laptop', 60000, '2023-07-24','South'],
    [19, 110, 'Mobile', 40000, '2023-07-08','West'],
    [20, 108, 'Tablet', 70000, '2023-03-18','East']
]
sales_schema=['sales_id','customer_id','product','amount','sale_date','region']
new_sales_df= spark.createDataFrame(data=new_sales_data,schema=sales_schema)

In [35]:
sales_df=sales_df.union(new_sales_df)
sales_df.show()

+--------+-----------+-------+------+----------+------+
|sales_id|customer_id|product|amount| sale_date|region|
+--------+-----------+-------+------+----------+------+
|       9|        104|Desktop| 45000|2023-08-10|  West|
|       3|        103| Tablet| 20000|2023-03-05|  West|
|       7|        102| Laptop| 60000|2023-06-15|  East|
|       6|        101| Mobile| 15000|2023-05-10| South|
|       1|        101| Laptop| 50000|2023-01-15| North|
|      10|        105| Laptop| 70000|2023-09-25| North|
|       4|        104| Laptop| 55000|2023-03-15|  East|
|       5|        105|Desktop| 40000|2023-04-20| North|
|       8|        103| Tablet| 20000|2023-07-05| North|
|       2|        102| Mobile| 15000|2023-02-10| South|
|      11|        114| Tablet| 70000|2023-09-13| North|
|      12|        115| Mobile| 50000|2023-11-16|  West|
|      13|        112| Tablet| 60000|2023-01-04| South|
|      14|        109|Desktop| 90000|2023-03-08|  East|
|      15|        108| Laptop| 80000|2023-06-25|

## 9. Window Functions


##### 29. Rank the sales records based on the amount column.


In [36]:
window2=Window.orderBy(col('amount').desc())
sales_df.withColumn('rank_on_amount',dense_rank().over(window2)).show()

# using Pyspark SQL queryy
print('using PySQL.....')
sales_df.createOrReplaceTempView('sales')
spark.sql('''
    select *,dense_rank() over(order by amount desc) as rank_of_amount
    from sales
''').show()


+--------+-----------+-------+------+----------+------+--------------+
|sales_id|customer_id|product|amount| sale_date|region|rank_on_amount|
+--------+-----------+-------+------+----------+------+--------------+
|      14|        109|Desktop| 90000|2023-03-08|  East|             1|
|      15|        108| Laptop| 80000|2023-06-25|  East|             2|
|      17|        115|Desktop| 80000|2023-02-05| North|             2|
|      10|        105| Laptop| 70000|2023-09-25| North|             3|
|      11|        114| Tablet| 70000|2023-09-13| North|             3|
|      20|        108| Tablet| 70000|2023-03-18|  East|             3|
|       7|        102| Laptop| 60000|2023-06-15|  East|             4|
|      13|        112| Tablet| 60000|2023-01-04| South|             4|
|      18|        112| Laptop| 60000|2023-07-24| South|             4|
|       4|        104| Laptop| 55000|2023-03-15|  East|             5|
|       1|        101| Laptop| 50000|2023-01-15| North|             6|
|     

##### 30. Add a cumulative sum of amount for each product in the sales DataFrame.


In [37]:
window3=Window.partitionBy('product').orderBy('sale_date')\
            .rowsBetween(Window.unboundedPreceding,Window.currentRow)
sales_df.withColumn('cummulative_sum',sum('amount').over(window3)).show()

+--------+-----------+-------+------+----------+------+---------------+
|sales_id|customer_id|product|amount| sale_date|region|cummulative_sum|
+--------+-----------+-------+------+----------+------+---------------+
|      17|        115|Desktop| 80000|2023-02-05| North|          80000|
|      14|        109|Desktop| 90000|2023-03-08|  East|         170000|
|       5|        105|Desktop| 40000|2023-04-20| North|         210000|
|       9|        104|Desktop| 45000|2023-08-10|  West|         255000|
|       1|        101| Laptop| 50000|2023-01-15| North|          50000|
|       4|        104| Laptop| 55000|2023-03-15|  East|         105000|
|       7|        102| Laptop| 60000|2023-06-15|  East|         165000|
|      15|        108| Laptop| 80000|2023-06-25|  East|         245000|
|      18|        112| Laptop| 60000|2023-07-24| South|         305000|
|      10|        105| Laptop| 70000|2023-09-25| North|         375000|
|       2|        102| Mobile| 15000|2023-02-10| South|         

##### 31. Add a column that calculates the difference between each customer's amount and the average amount within their product group.

In [38]:
window4=Window.partitionBy('product')
sales_df.withColumn('avg_amount',avg('amount').over(window4))\
        .withColumn('difference_amount_and_avg',col('amount')-col('avg_amount'))\
        .show()

+--------+-----------+-------+------+----------+------+------------------+-------------------------+
|sales_id|customer_id|product|amount| sale_date|region|        avg_amount|difference_amount_and_avg|
+--------+-----------+-------+------+----------+------+------------------+-------------------------+
|       9|        104|Desktop| 45000|2023-08-10|  West|           63750.0|                 -18750.0|
|       5|        105|Desktop| 40000|2023-04-20| North|           63750.0|                 -23750.0|
|      14|        109|Desktop| 90000|2023-03-08|  East|           63750.0|                  26250.0|
|      17|        115|Desktop| 80000|2023-02-05| North|           63750.0|                  16250.0|
|       7|        102| Laptop| 60000|2023-06-15|  East|           62500.0|                  -2500.0|
|       1|        101| Laptop| 50000|2023-01-15| North|           62500.0|                 -12500.0|
|      10|        105| Laptop| 70000|2023-09-25| North|           62500.0|                 

## 10. Partitioning


##### 32. Write the sales DataFrame to a partitioned Parquet file by region.


In [39]:
sales_df.write.partitionBy('region')\
    .mode('overwrite')\
    .parquet('sales.parquet')

##### 33. Partition the customer DataFrame by customer_city and save it as a CSV file.


In [40]:
customer_df.write.partitionBy('customer_city')\
    .mode('append')\
    .csv('customers.csv')

## 11. Real-World Scenarios


##### 34. Calculate the percentage contribution of each product to the total sales.


In [41]:
total_sales=sales_df.groupBy().sum('amount').collect()[0][0]
sales_df.groupBy('product').agg((sum('amount')*100/total_sales).alias('contribution_perc'))\
    .show()

+-------+------------------+
|product| contribution_perc|
+-------+------------------+
| Laptop| 36.05769230769231|
| Mobile|11.538461538461538|
| Tablet|27.884615384615383|
|Desktop| 24.51923076923077|
+-------+------------------+



##### 35. Extract the year from sale_date and group by year to calculate total sales.


In [42]:
sales_df.groupBy(year('sale_date').alias('year')).agg(sum('amount').alias('total_sales')).show()

+----+-----------+
|year|total_sales|
+----+-----------+
|2023|    1040000|
+----+-----------+



##### 36. Identify the most purchased product in each region.


In [43]:
sales_new=sales_df.groupBy('region','product').agg(count('product').alias('prod_count'))
window5=Window.partitionBy('region').orderBy(col('prod_count').desc())
sales_new.withColumn('most_purchased',dense_rank().over(window5))\
        .filter(col('most_purchased')==1)\
        .select('region','product')\
        .show()

+------+-------+
|region|product|
+------+-------+
|  East| Laptop|
| North| Tablet|
| South| Mobile|
|  West| Mobile|
+------+-------+



##### 37. Add a column to show the difference between the highest and lowest sales for each product

In [44]:
sales_df.groupBy('product').agg(min('amount').alias('lowest_sale'),max('amount').alias('highest_sale'))\
    .withColumn('difference',col('highest_sale')-col('lowest_sale'))\
    .show()

# using PySQL
print('using PySQL....')
sales_df.createOrReplaceTempView('sales')
spark.sql('''
        select product,max(amount)-min(amount) as difference
        from sales
        group by product
''').show()

+-------+-----------+------------+----------+
|product|lowest_sale|highest_sale|difference|
+-------+-----------+------------+----------+
| Laptop|      50000|       80000|     30000|
| Mobile|      15000|       50000|     35000|
| Tablet|      20000|       70000|     50000|
|Desktop|      40000|       90000|     50000|
+-------+-----------+------------+----------+

using PySQL....
+-------+----------+
|product|difference|
+-------+----------+
| Laptop|     30000|
| Mobile|     35000|
| Tablet|     50000|
|Desktop|     50000|
+-------+----------+



##### 38. Write the result of the join between sales and customer to parquet file.


In [45]:
sale_customer_joined=sales_df.join(customer_df,sales_df['customer_id']==customer_df['customer_id'],'inner')\
    .select('sales_id','product','amount','sale_date','region',sales_df['customer_id'],'customer_name','email','age','customer_city')
sale_customer_joined.show()
sale_customer_joined.write.mode('overwrite').parquet('sales_customer_joined.parquet')

+--------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|sales_id|product|amount| sale_date|region|customer_id|customer_name|               email|age|customer_city|
+--------+-------+------+----------+------+-----------+-------------+--------------------+---+-------------+
|       9|Desktop| 45000|2023-08-10|  West|        104|  Priya Patel|priya.patel@email...| 27|    Ahmedabad|
|       3| Tablet| 20000|2023-03-05|  West|        103|  Rahul Yadav|rahul.yadav@email...| 30|    Bangalore|
|       7| Laptop| 60000|2023-06-15|  East|        102|  Meena Verma|meena.verma@email...| 34|       Mumbai|
|       6| Mobile| 15000|2023-05-10| South|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|       1| Laptop| 50000|2023-01-15| North|        101|  Arun Sharma|arun.sharma@email...| 28|        Delhi|
|      10| Laptop| 70000|2023-09-25| North|        105|  Sneha Reddy|sneha.reddy@email...| 29|    Hyderabad|
|       4| Laptop| 

##### 39. Identify products that were sold in the last 6 months.


In [46]:
sales_df.withColumn('months_diff',months_between(current_date(),col('sale_date')))\
    .filter(col('months_diff')<=6)\
    .show()

+--------+-----------+-------+------+---------+------+-----------+
|sales_id|customer_id|product|amount|sale_date|region|months_diff|
+--------+-----------+-------+------+---------+------+-----------+
+--------+-----------+-------+------+---------+------+-----------+



##### 40. Calculate the average sales amount per customer.

In [47]:
sales_df.groupBy('customer_id').agg(avg('amount').alias('avg_amount_per_customer'))\
    .orderBy('customer_id')\
    .show()

# 
print('using PySQL....')
spark.sql('''
    select customer_id,avg(amount) as avg_sale
    from sales
    group by customer_id
    order by customer_id
''').show()

+-----------+-----------------------+
|customer_id|avg_amount_per_customer|
+-----------+-----------------------+
|        101|                32500.0|
|        102|                37500.0|
|        103|                20000.0|
|        104|                50000.0|
|        105|                55000.0|
|        108|                75000.0|
|        109|                90000.0|
|        110|                40000.0|
|        112|                60000.0|
|        113|                50000.0|
|        114|                70000.0|
|        115|                65000.0|
+-----------+-----------------------+

using PySQL....
+-----------+--------+
|customer_id|avg_sale|
+-----------+--------+
|        101| 32500.0|
|        102| 37500.0|
|        103| 20000.0|
|        104| 50000.0|
|        105| 55000.0|
|        108| 75000.0|
|        109| 90000.0|
|        110| 40000.0|
|        112| 60000.0|
|        113| 50000.0|
|        114| 70000.0|
|        115| 65000.0|
+-----------+--------+

